In [ ]:
!pip install tensorflow_text -q

     |████████████████████████████████| 4.9 MB 5.3 MB/s 


In [ ]:
import os
import requests
import time

In [1]:
import pandas as pd

In [2]:
df = pd.read_pickle('../python_dataset.pkl')

In [5]:
pd.set_option('display.max_column', None)

In [6]:
df.head(5)

,code,docstring,code_tokens,docstring_tokens,partition
0,def zmq_device(self):\n '''\n Mu...,Multiprocessing target for the zmq queue device,"[def, zmq_device, (, self, ), :, self, ., __se...","[Multiprocessing, target, for, the, zmq, queue...",train
1,def close(self):\n '''\n Cleanly...,Cleanly shutdown the router socket,"[def, close, (, self, ), :, if, self, ., _clos...","[Cleanly, shutdown, the, router, socket]",train
2,"def pre_fork(self, process_manager):\n ...",Pre-fork we need to create the zmq router devi...,"[def, pre_fork, (, self, ,, process_manager, )...","[Pre, -, fork, we, need, to, create, the, zmq,...",train
3,def _start_zmq_monitor(self):\n '''\n ...,Starts ZMQ monitor for debugging purposes.\n ...,"[def, _start_zmq_monitor, (, self, ), :, # Soc...","[Starts, ZMQ, monitor, for, debugging, purpose...",train
4,"def post_fork(self, payload_handler, io_loop):...",After forking we need to create all of the loc...,"[def, post_fork, (, self, ,, payload_handler, ...","[After, forking, we, need, to, create, all, of...",train


In [ ]:
from zipfile import ZipFile
import gzip
import json

In [ ]:
import tensorflow as tf
from tensorflow.data import Dataset
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab
import tensorflow_text as text

import pandas as pd

In [ ]:
project_id = 'firm-foundation-331820'
BUCKET_NAME = 'code-search-python-dataset'

In [ ]:
pd.set_option('display.max_column', None)

In [ ]:
def extract_file(file_name):
  base_file_name = file_name.split('.')[0]

  with ZipFile(file_name, mode='r') as zip_obj:
    os.makedirs(base_file_name)
    os.chdir(base_file_name)

    zip_obj.extractall()

    os.chdir('../')


In [ ]:
def download_file(base_url, file_name):
  with requests.get(base_url + file_name, stream=False) as r:
    with open(file_name, mode='wb') as f:
      f.write(r.content)

Download and extract all.

In [ ]:
base_url = 'https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/'
languages = ['python']
extension = '.zip'

In [ ]:
for language in languages:
  file_name = language + extension
  download_file(base_url, file_name=file_name)
  extract_file(file_name)

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
language = 'python'
base_file_path = f'{language}/{language}/final/jsonl/'
columns = ['code', 'docstring', 'code_tokens', 'docstring_tokens', 'partition']

df_train_python = pd.concat([pd.read_json((base_file_path + dir + '/' + relative_path), compression='gzip', 
                                          lines=True)[columns] 
                             for dir in ['train', 'valid', 'test'] 
                             for relative_path in os.listdir(base_file_path + dir + '/')])

df_train_python.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 457461 entries, 0 to 22175
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   code              457461 non-null  object
 1   docstring         457461 non-null  object
 2   code_tokens       457461 non-null  object
 3   docstring_tokens  457461 non-null  object
 4   partition         457461 non-null  object
dtypes: object(5)
memory usage: 20.9+ MB


In [ ]:
df_train_python.to_pickle('/content/python_dataset.pkl')

Uploading to GCS for training.

In [ ]:
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
!gsutil cp python_dataset.pkl gs://{BUCKET_NAME}/

Copying file://python_dataset.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\
Operation completed over 1 objects/1.0 GiB.                                      


In [ ]:
!gsutil ls gs://{BUCKET_NAME}

gs://code-search-python-dataset/python_dataset.pkl


In [ ]:
def data_gen_pandas_for_vocab():
  for index, row in df_train_python.iterrows():
    yield row['code'], row['docstring']

In [ ]:
vocab_dataset = Dataset.from_generator(data_gen_pandas_for_vocab, 
                               output_signature=(tf.TensorSpec(shape=(), dtype=tf.string),
                                                 tf.TensorSpec(shape=(), dtype=tf.string)))

In [ ]:
tokenizer_param = {'lower_case' : True}
reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]", "[SEP]"]
vocab_args = {
    'vocab_size' : 33000,
    'reserved_tokens' : reserved_tokens,
    'bert_tokenizer_params' : tokenizer_param,
    'learn_params' : {}
}

In [ ]:
code_ds = vocab_dataset.map(lambda code, _: code).batch(10000).prefetch(5)
docstring_ds = vocab_dataset.map(lambda _, docstring: docstring).batch(10000).prefetch(5)

In [ ]:
def write_vocab_file(filename, vocab):
  with open(filename, mode='w') as f:
    for word in vocab:
      print(word, file=f)

Creating vocabulary for code and docstring tokenizers using WordPiece Algorithm.

In [ ]:
code_vocab = bert_vocab.bert_vocab_from_dataset(dataset=code_ds, **vocab_args)
write_vocab_file('code_vocab.txt', code_vocab)

In [ ]:
docstring_vocab = bert_vocab.bert_vocab_from_dataset(dataset=docstring_ds, **vocab_args)
write_vocab_file('docstring_vocab.txt', docstring_vocab)

Tokenizers each having their own vocabulary.

In [ ]:
code_tokenizer = text.BertTokenizer('code_vocab.txt', **tokenizer_param)
docstring_tokenizer = text.BertTokenizer('docstring_vocab.txt', **tokenizer_param)